In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('Absenteeism_preprocessed.csv')
data.head(10)

,GRP1,GRP2,GRP3,GRP4,Month Value,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2
5,0,0,0,1,10,2,179,51,38,239.554,31,0,0,0,2
6,0,0,0,1,7,4,361,52,28,239.554,27,0,1,4,8
7,0,0,0,1,7,4,260,50,36,239.554,23,0,4,0,4
8,0,0,1,0,6,6,155,12,34,239.554,25,0,2,0,40
9,0,0,0,1,7,0,235,11,37,239.554,29,1,1,1,8


In [3]:
targets=np.where(data['Absenteeism Time in Hours']>data['Absenteeism Time in Hours'].median(),1,0)

In [4]:
data['Targets']=targets
tdata=data.drop({'Absenteeism Time in Hours','Day of Week',
                 'Daily Work Load Average','Distance to Work'},axis=1)

In [5]:
unscaled_input=tdata.iloc[:,:-1]

# standardization


In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [7]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import StandardScaler

In [8]:
class CustomScaler(BaseEstimator,TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler=StandardScaler(copy,with_mean,with_std)
        self.columns=columns
        self.mean_=None
        self.var_=None
        
    def fit(self,X,y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_ =np.mean(X[self.columns])
        self.var_ =np.var(X[self.columns])
        return self
    
    def transform(self,X,y=None,copy=None):
        init_col_order=X.columns
        X_scaled=pd.DataFrame(self.scaler.transform(X[self.columns]),columns=self.columns)
        X_not_scaled=X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled],axis=1)[init_col_order]

In [9]:
unscaled_input.columns.values

array(['GRP1', 'GRP2', 'GRP3', 'GRP4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [10]:
column_to_scale=['Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index',
       'Children', 'Pets']

In [11]:
scaler=  CustomScaler(column_to_scale)

/Users/harshgupta/opt/anaconda3/envs/py3-TF2.0/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass copy=True, with_mean=True, with_std=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [12]:
scaler.fit(unscaled_input)

AttributeError: 'CustomScaler' object has no attribute 'copy'

AttributeError: 'CustomScaler' object has no attribute 'copy'

In [13]:
scaled_input=scaler.transform(unscaled_input)

In [14]:
scaled_input

,GRP1,GRP2,GRP3,GRP4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.030796,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.030796,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.030796,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.030796,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.030796,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.568019,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.568019,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.568019,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.568019,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


# Split into train test


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(scaled_input,targets,train_size=0.8,random_state=20)

In [17]:
print(x_train.shape,y_train.shape)

(560, 11) (560,)


# logistic regression

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
reg=LogisticRegression()

In [20]:
reg.fit(x_train,y_train)

LogisticRegression()

In [21]:
reg.score(x_train,y_train)

0.7767857142857143

In [22]:
reg.intercept_

array([-1.60957471])

In [23]:
feature_name=unscaled_input.columns.values

In [24]:
summary_table=pd.DataFrame(columns=['Feature Name'],data=feature_name)
summary_table['Coefficient']=np.transpose(reg.coef_)
summary_table

,Feature Name,Coefficient
0,GRP1,2.771512
1,GRP2,0.931688
2,GRP3,3.092102
3,GRP4,0.809059
4,Month Value,0.007812
5,Transportation Expense,0.625055
6,Age,-0.173903
7,Body Mass Index,0.288294
8,Education,-0.240816
9,Children,0.357535


In [25]:
summary_table.index=summary_table.index + 1
summary_table.loc[0]=['Intercept',reg.intercept_[0]]
summary_table=summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-1.609575
1,GRP1,2.771512
2,GRP2,0.931688
3,GRP3,3.092102
4,GRP4,0.809059
5,Month Value,0.007812
6,Transportation Expense,0.625055
7,Age,-0.173903
8,Body Mass Index,0.288294
9,Education,-0.240816


In [26]:
summary_table['Odds']=np.exp(summary_table.Coefficient)

In [27]:
summary_table.sort_values('Odds',ascending=False)

,Feature Name,Coefficient,Odds
3,GRP3,3.092102,22.023327
1,GRP1,2.771512,15.982778
2,GRP2,0.931688,2.538791
4,GRP4,0.809059,2.245794
6,Transportation Expense,0.625055,1.868348
10,Children,0.357535,1.429801
8,Body Mass Index,0.288294,1.334150
5,Month Value,0.007812,1.007843
7,Age,-0.173903,0.840378
9,Education,-0.240816,0.785986


# TEST

In [28]:
reg.score(x_test,y_test)

0.7357142857142858

# SAVE

In [29]:
import pickle 

with open('model','wb') as file:
    pickle.dump(reg,file)
with open('scaler','wb') as file:
    pickle.dump(scaler,file)